##Notebook for getting summary breakdowns of habitat occurance on Defra Group Land

Miles Clement, Feb 2025

**NOTE:** Area calculations are based on model outputs, and not the full vector geometries. The values calculated for Defra Land will vary from those previously reported in Phase 1 due to this generalised representation of the spatial extent.

**NOTE 2:** There is overlap between the extent of freehold and leasehold land parcels. The habitat extent of these will be reported separately.

**Habitat Notes**
- Saltmarsh is a subset of Coastal Margins, and included in the extent of the latter (beware of double counting)
- Upland Bog is a subset of Moorland & Heath, and included in the extent of the latter (beware of double counting)
- Dense and Sparse Woodland will also be reported as combined/mixed woodland

---------
###SETUP
####Load Packages

In [0]:
import pandas as pd
import os
from pathlib import Path
from functools import reduce
from sds_dash_download import download_file

In [0]:
from sedona.spark import *
from pyspark.sql.functions import expr, when, col, lit, sum
from pyspark.sql import functions as F

sedona = SedonaContext.create(spark)
sqlContext.clearCache()

username = (
    dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
)

####User Input Variables

In [0]:
### UPDATE THIS CELL ###
# Combined Asset Table Directory
asset_table_dir = Path('/dbfs/mnt/lab-res-a1001005/esd_project/Defra_Land/Final/Asset_Tables')
# Out Directory
stats_out_dir = Path('/dbfs/mnt/lab-res-a1001005/esd_project/Defra_Land/Final/Stats')

In [0]:
# Define asset columns that represent each habitat
habitats_cols = {
    'mixed_woodland': ["le_comb","lcm_comb","nfi_dense","phi_deciduous_woodland","le_scrub","nfi_sparse","wood_pasture_park","phi_traditional_orchard","fr_tow"],
    'dense_woodland': ["le_comb","lcm_comb","nfi_dense","phi_deciduous_woodland"],
    'sparse_woodland': ["le_scrub","nfi_sparse","wood_pasture_park","phi_traditional_orchard","fr_tow"],
    'moorland': ["le_comb","phi_comb","lcm_comb"],
    'upland_bog': ["le_bog","phi_blanket_bog","lcm_bog"],
    'grassland': ["le_unimproved_grass","phi_comb","lcm_comb"],
    'coastal': ["le_comb","phi_comb","lcm_comb","ne_marine"],
    'saltmarsh': ["le_saltmarsh","phi_saltmarsh","lcm_saltmarsh","ne_marine_saltmarsh"],
    'arable': ['le_comb', 'phi_comb', 'lcm_comb', 'crome'],
    'water': ["le_comb","lcm_comb","phi_comb","os_ngd_water"],
    'urban': ["le_urban","lcm_comb","ons_urban"]
}

In [0]:
# Derivatives
alt_table_dir = str(asset_table_dir).replace("/dbfs", "dbfs:")
parquet_files = os.listdir(asset_table_dir)

#### Load in Core Datasets

In [0]:
# Load in an example asset table for calculating national scale statistics
dgl_data_example = sedona.read.format("parquet").load(f"{alt_table_dir}/{parquet_files[0]}")

# Load in Phase 1 DGL outputs to calc overall difference in extent
dgl_phase1 = sedona.read.format("parquet").load("dbfs:/mnt/lab-res-a1001005/esd_project/jasmine.elliott@defra.gov.uk/gov_land_analysis/phase_one_final_report_outputs/polygon_ccod_defra_by_organisation_tenure.parquet")

# Load in ownership data for Defra Land
dgl_ownership = sedona.read.format("parquet").load("dbfs:/mnt/lab-res-a1001005/esd_project/Defra_Land/Assets/10m_x_dgl_organisation.parquet")

####Functions

In [0]:
def calc_ha(tenure,
            data_in, 
            row,
            hab_condition, 
            tenure_condition,
            org_condition = None):
  
  """Combine condition arguments and calculate area in ha for different Defra land tenures & organisations.

    Args:
          tenure (string): Type of land tenure, either 'freehold', 'leasehold' or 'mixed'
          data_in (spark dataframe): Main dataset, table of boolean indicators for specific habitat
          row (dictionary): Output dictionary entry to be added to
          hab_condition (pyspark column object): List of columns as a boolean condition for analysing data_in, representing habitat indicators
          tenure_condition (pyspark column object): List of columns as a boolean condition for analysing data_in, representing tenure
          organisation_condition (pyspark column object, default None): List of columns as a boolean condition for analysing data_in, representing organisation indicators
    """ 
  
  if org_condition is None:
    full_condition = tenure_condition & hab_condition
  elif org_condition is not None:
    full_condition = tenure_condition & hab_condition & org_condition

  n = data_in.filter(full_condition).count()
  ha = n / 100
  
  if tenure == 'freehold':
    row[f'{tenure}_ha'] = ha
  elif tenure == 'leasehold':
    row[f'{tenure}_ha'] = ha
  elif tenure == 'mixed':
    row[f'{tenure}_tenure_ha'] = ha

  return row

#### Constants & High-Level Stats
- Overall Extent
- Difference between model representation of DGL and the vector input

In [0]:
# Get row counts for each split of data (by tenure)
leasehold_count = dgl_data_example.filter((dgl_data_example.dgl_fh.isNull()) & (dgl_data_example.dgl_lh == 1)).count()
freehold_count = dgl_data_example.filter((dgl_data_example.dgl_fh == 1) & (dgl_data_example.dgl_lh.isNull())).count()
both_count = dgl_data_example.filter((dgl_data_example.dgl_fh == 1) & (dgl_data_example.dgl_lh == 1)).count() 
total_count = dgl_data_example.filter((dgl_data_example.dgl_fh == 1) | (dgl_data_example.dgl_lh == 1)).count()

###Logic of Maths
- Each '1' in a column represents overlap with the centroid of a 10x10m cell
- The area of each cell is 100m2
- To convert from m2 into ha, divide by 10,000
- To calculate the total area of the count of columns, multiply by 100 (m2 area) and divide by 10,000 (ha area)
- This distills down to dividing the number of cells by 100

In [0]:
leasehold_ha = leasehold_count / 100
freehold_ha = freehold_count / 100
both_ha = both_count / 100
total_ha = total_count / 100

In [0]:
# Print model area totals for whole of Defra land
print(f"Leasehold: {leasehold_ha} ha")
print(f"Freehold: {freehold_ha} ha")
print(f"Both holding types: {both_ha} ha")
print(f"Total: {total_ha} ha")

In [0]:
# Calculate the total area of the vector dataset
phase1_area = dgl_phase1.agg(sum("area_ha").alias("total_area")).collect()[0]["total_area"]

In [0]:
# Print model and vector area totals as comparison of accuracy of model outputs
print(f"Model Area: {total_ha} ha")
print(f"Vector Area: {round(phase1_area,1)} ha")
print(f"Difference: {round(phase1_area-total_ha,1)} ha more in the vector dataset")
print(f"This equates to {round(((phase1_area-total_ha)/phase1_area)*100,2)}% of the vector dataset area")

####Table 1: Habitat broken down by Leasehold/Freehold/Both

In [0]:
results_tenure = []

In [0]:
# Iterate habitat parquet files
for habitat, indicators in habitats_cols.items():

  print(habitat)

  row = {"habitat": habitat} 

  data_in = sedona.read.format("parquet").load(f"{alt_table_dir}/10m_x_assets_combined_{habitat}.parquet")

  # Set indicator columns for checking 
  hab_condition = reduce(
    lambda acc, col: acc | (data_in[col] == 1),
    indicators,
    lit(False))
    
  # Additional condiiton for upland bog (above moorland line)
  if habitat == "upland_bog":
    moorland_line_condition = col("moorland_line") == 1
    hab_condition = moorland_line_condition & hab_condition  

  # Iterate freehold/leasehold/both
  for tenure in ['dgl_fh','dgl_lh','mixed']:

    if tenure == 'dgl_fh':
      tenure_condition = ((data_in[tenure] == 1) & (data_in['dgl_lh'].isNull()))
      row = calc_ha('freehold', data_in, row, hab_condition, tenure_condition)

    elif tenure == 'dgl_lh':
      tenure_condition = ((data_in[tenure] == 1) & (data_in['dgl_fh'].isNull()))
      row = calc_ha('leasehold', data_in, row, hab_condition, tenure_condition)

    elif tenure == 'mixed':
      tenure_condition = ((data_in['dgl_fh'] == 1) & (data_in['dgl_lh'] == 1))
      row = calc_ha('mixed', data_in, row, hab_condition, tenure_condition)
 
  results_tenure.append(row)

In [0]:
summary_tenure = pd.DataFrame(results_tenure)

In [0]:
# Calc various % of model total DGL and tenure-specific extens
summary_tenure['total_ha'] = summary_tenure['freehold_ha'] + summary_tenure['leasehold_ha'] + summary_tenure['mixed_tenure_ha'] # Includes overlap between tenures
summary_tenure['fh_%_dgl_all'] = ((summary_tenure['freehold_ha']/ total_ha) *100).round(2)
summary_tenure['fh_%_dgl_fh'] = ((summary_tenure['freehold_ha']/ freehold_ha) *100).round(2)
summary_tenure['lh_%_dgl_all'] = ((summary_tenure['leasehold_ha']/ total_ha) *100).round(2)
summary_tenure['lh_%_dgl_lh'] = ((summary_tenure['leasehold_ha']/ leasehold_ha) *100).round(2)
summary_tenure['mix_%_dgl_all'] = ((summary_tenure['mixed_tenure_ha']/ total_ha) *100).round(2)
summary_tenure['all_%_dgl_all'] = ((summary_tenure['total_ha']/ total_ha) *100).round(2)

In [0]:
summary_tenure

In [0]:
# Calc % of DGL from 'total_ha' as a proxy for double counting
excluded_habitats = ["saltmarsh", "upland_bog","dense_woodland","sparse_woodland"] # Excluded as double counted habitats
summary_subsets_rm = summary_tenure[~summary_tenure['habitat'].isin(excluded_habitats)]
tabulated_perc = summary_subsets_rm['all_%_dgl_all'].sum().round(2)
tabulated_ha = summary_subsets_rm['total_ha'].sum().round(2)
print(f'% of DGL covered by habitat extents: {tabulated_perc}')
print(f'This means {(tabulated_ha-total_ha).round(2)} ha of land with overlap across multiple habitats')

In [0]:
summary_tenure.to_csv(f'{stats_out_dir}/dgl_summary_tenure.csv')

In [0]:
displayHTML(download_file(f'{stats_out_dir}/dgl_summary_tenure.csv', move=False))

####Table 2: Habitat broken down by Leasehold/Freehold & Owner Organisation

In [0]:
# Extract unique organisations from ownership data
unique_orgs = dgl_ownership.select("organisation").dropDuplicates()
unique_orgs = unique_orgs.rdd.flatMap(lambda x: x).collect()
unique_orgs

In [0]:
results_org = []

In [0]:
# Iterate habitat parquet files
for habitat, indicators in habitats_cols.items():

  print(habitat)

  data_in = sedona.read.format("parquet").load(f"{alt_table_dir}/10m_x_assets_combined_{habitat}.parquet")

  data_org = data_in.join(dgl_ownership, on="id", how="left")

  # Set indicator columns for checking 
  hab_condition = reduce(
    lambda acc, col: acc | (data_in[col] == 1),
    indicators,
    lit(False))
    
  # Additional condiiton for upland bog (above moorland line)
  if habitat == "upland_bog":
    moorland_line_condition = col("moorland_line") == 1
    hab_condition = moorland_line_condition & hab_condition  

  for org in unique_orgs:
    
    row = {"habitat": habitat} 
    row["organisation"] = org

    org_condition = (data_org['organisation'] == org)

    # Iterate freehold/leasehold/both
    for tenure in ['dgl_fh','dgl_lh','mixed']:

      if tenure == 'dgl_fh':
        tenure_condition = ((data_in[tenure] == 1) & (data_in['dgl_lh'].isNull()))
        row = calc_ha('freehold', data_org, row, hab_condition, tenure_condition, org_condition)

      elif tenure == 'dgl_lh':
        tenure_condition = ((data_in[tenure] == 1) & (data_in['dgl_fh'].isNull()))
        row = calc_ha('leasehold', data_org, row, hab_condition, tenure_condition, org_condition)

      elif tenure == 'mixed':
        tenure_condition = ((data_in['dgl_fh'] == 1) & (data_in['dgl_lh'] == 1))
        row = calc_ha('mixed', data_org, row, hab_condition, tenure_condition, org_condition)
 
    results_org.append(row)

In [0]:
summary_org = pd.DataFrame(results_org)

# Remove any habitat-organisation pairs that have no land in any of the columns
summary_org = summary_org[~((summary_org['freehold_ha'] == 0) & (summary_org['leasehold_ha'] == 0) & (summary_org['mixed_tenure_ha'] == 0))]

summary_org

In [0]:
summary_org.to_csv(f'{stats_out_dir}/dgl_summary_organisation.csv')

In [0]:
displayHTML(download_file(f'{stats_out_dir}/dgl_summary_organisation.csv', move=False))

###Plotting Play

In [0]:
summary_org["organisation"] = summary_org["organisation"].replace(
    "Department for Environment, Food and Rural Affairs (Forestry Commission or Forestry England)", "DEFRA (FC/FE)"
)


In [0]:
# Define the list of habitats to exclude
excluded_habitats = ["dense_woodland", "sparse_woodland	", "saltmarsh","upland_bog"]

# Filter out rows with any of the excluded habitats
filtered_summary = summary_org[~summary_org["habitat"].isin(excluded_habitats)]

In [0]:
summary_df = filtered_summary.groupby("organisation", as_index=False).agg(
    {
        "freehold_ha": "sum",
        "leasehold_ha": "sum",
        "mixed_tenure_ha": "sum",
    }
)

summary_df["total_ha"] = summary_df["freehold_ha"] + summary_df["leasehold_ha"] + summary_df["mixed_tenure_ha"]

summary_df = summary_df.sort_values(by="total_ha", ascending=False)

summary_df = summary_df.reset_index(drop=True)

summary_df["organisation"] = summary_df["organisation"].replace(
    "Department for Environment, Food and Rural Affairs (Forestry Commission or Forestry England)", "DEFRA (FC/FE)"
)

summary_df

In [0]:
top_3_organisations = summary_df.head(3)["organisation"].tolist()

In [0]:
top_3_organisations

In [0]:
top_3_data = summary_org[summary_org["organisation"].isin(top_3_organisations)]

# Group by organisation and habitat, then sum the relevant columns
habitat_summary = top_3_data.groupby(["organisation", "habitat"], as_index=False).agg(
    {
        "freehold_ha": "sum",
        "leasehold_ha": "sum",
        "mixed_tenure_ha": "sum",
    }
)

# Add a column for the total area
habitat_summary["total_ha"] = (
    habitat_summary["freehold_ha"]
    + habitat_summary["leasehold_ha"]
    + habitat_summary["mixed_tenure_ha"]
)

# Display the result
habitat_summary

In [0]:
# Merge habitat_summary with summary_df to get the total_ha for each organisation
habitat_summary = habitat_summary.merge(summary_df[["organisation", "total_ha"]], on="organisation", suffixes=("", "_org_total"))

# Calculate the percentage of total_ha for each habitat
habitat_summary["percentage_of_total"] = (habitat_summary["total_ha"] / habitat_summary["total_ha_org_total"]) * 100

# Select only the desired columns
habitat_summary = habitat_summary[["organisation", "habitat", "percentage_of_total"]]

# Display the updated DataFrame
habitat_summary


In [0]:
habitat_summary["organisation"] = habitat_summary["organisation"].replace(
    "Department for Environment, Food and Rural Affairs (Forestry Commission or Forestry England)", "DEFRA (FC/FE)"
)

In [0]:
habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "arable", "Enclosed Farmland"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "coastal", "Marine & Coastal Margins"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "dense_woodland", "Woodland (Dense)"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "grassland", "Semi-Natural Grassland"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "mixed_woodland", "Woodland (Mixed)"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "moorland", "Mountain, Moorland & Heath"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "saltmarsh", "Saltmarsh"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "sparse_woodland", "Woodland (Sparse)"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "upland_bog", "Upland Bog"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "urban", "Urban"
)

habitat_summary["habitat"] = habitat_summary["habitat"].replace(
    "water", "Freshwater & Wetlands"
)

habitat_summary

In [0]:
organisation_order = ['DEFRA (FC/FE)', 'Natural England', 'Environment Agency']

habitat_summary['organisation'] = pd.Categorical(habitat_summary['organisation'], categories=organisation_order, ordered=True)

habitat_summary = habitat_summary.sort_values(by=['organisation', 'habitat'])

habitat_summary

In [0]:
plot_habs = ["Woodland (Mixed)", "Woodland (Dense)","Woodland (Sparse)","Mountain, Moorland & Heath","Upland Bog","Semi-Natural Grassland","Marine & Coastal Margins","Saltmarsh","Enclosed Farmland","Freshwater & Wetlands", "Urban"]  


In [0]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

# Define the habitat colors with fill, edge, and pattern
habitat_colors = {
    "Woodland (Mixed)": {"fill": "#28A197", "edge": "#28A197", "pattern": None},
    "Woodland (Dense)": {"fill": "#BFBFBF", "edge": "#28A197", "pattern": "//"},
    "Woodland (Sparse)": {"fill": "#BFBFBF", "edge": "#28A197", "pattern": "\\"},
    "Mountain, Moorland & Heath": {"fill": '#A285D1', "edge": "#A285D1", "pattern": None},
    "Upland Bog": {"fill": '#BFBFBF', "edge": "#A285D1", "pattern": "O"},
    "Semi-Natural Grassland": {"fill": '#F46A25', "edge": "#F46A25", "pattern": None},
    "Marine & Coastal Margins": {"fill": '#12436D', "edge": "#12436D", "pattern": None},
    "Saltmarsh": {"fill": '#BFBFBF', "edge": "#12436D", "pattern": "."},
    "Enclosed Farmland": {"fill": '#801650', "edge": "#801650", "pattern": None},
    "Freshwater & Wetlands": {"fill": '#2073BC', "edge": "#2073BC", "pattern": None},
    "Urban": {"fill": '#3D3D3D', "edge": "#3D3D3D", "pattern": None}
}

# Generate the habitat fill colors list from the dictionary
habitat_fill_colors = [habitat_colors[hab]["fill"] for hab in plot_habs]

# Define the order of organisations
organisation_order = ['DEFRA (FC/FE)', 'Natural England', 'Environment Agency']

# Initialize figure
plt.figure(figsize=(18, 11))

# Sort the habitat_summary dataframe based on the organisation_order
habitat_summary = habitat_summary[habitat_summary['organisation'].isin(organisation_order)]
habitat_summary['organisation'] = pd.Categorical(habitat_summary['organisation'], categories=organisation_order, ordered=True)

# Create a list to store the positions of the bars (the x-position for each group)
bar_width = 0.07  # Adjust the width of the bars
x_positions = np.arange(len(organisation_order))  

# Loop through each habitat and plot
for i, habitat in enumerate(plot_habs):
    # Filter data for the current habitat
    habitat_data = habitat_summary[habitat_summary['habitat'] == habitat]
    
    # Calculate the positions for this habitat's bars
    positions = x_positions + i * bar_width  # Slightly offset each habitat's bars
    
    # Get the color and pattern for the current habitat
    habitat_color = habitat_colors[habitat]
    fill_color = habitat_color["fill"]
    edge_color = habitat_color["edge"]
    pattern = habitat_color["pattern"]
    
    # Plot the bars for this habitat
    bars = plt.bar(
        positions,
        habitat_data['percentage_of_total'],
        width=bar_width,
        label=habitat,
        color=fill_color,
        edgecolor=edge_color,
        zorder=3
    )
    
    # Apply pattern if defined
    if pattern:
        for bar in bars:
            bar.set_hatch(pattern)

# Add the total_ha labels to the x-axis
# First, get the total_ha for each organisation
organisation_names = habitat_summary['organisation'].unique()
total_ha_values = [summary_df.loc[summary_df['organisation'] == org, 'total_ha'].values[0] for org in organisation_names]

# Create a list with both organisation names and formatted total_ha (with commas and no parentheses)
x_labels = [f"{total_ha:,.1f} ha" for org, total_ha in zip(organisation_names, total_ha_values)]

# Add labels and title
plt.ylabel("Percentage of Organisation Total (%)", fontsize=21)
plt.xticks(x_positions + (len(plot_habs) * bar_width) / 2, organisation_names, fontsize=21)

# Adjust the positioning of the total_ha labels (move them below the organisations)
for i, label in enumerate(x_labels):
    plt.text(
        x_positions[i] + (len(plot_habs) * bar_width) / 2,  # x-position (aligned with the bar)
        -4.4,  # y-position (move down below the x-axis)
        label,  # The label content
        ha='center',  # Horizontal alignment
        fontsize=21,  # Font size for the labels
        color='black'  # Color for the labels
    )

plt.yticks(fontsize=18)
plt.legend(fontsize=20)
plt.tight_layout()
plt.grid(axis='y', zorder=0)

# Show the plot
plt.show()





In [0]:
import sys
print(sys.version)